<a href="https://colab.research.google.com/github/DARoSLab/Adv_robot_dyn_ctrl/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4. Physics Simulation


$\color{red}{\text{Please do not use any other Python packages except the ones we give you.}}$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import seaborn as sns
from matplotlib import animation
from IPython.display import HTML
%matplotlib inline

# Q.1 [20 pts] **Ball Bouncing Simulation**
<img src="https://github.com/DARoSLab/Adv_robot_dyn_ctrl/blob/main/Figures/bouncing_ball.png?raw=true" width="400">


## **Q.1 (a) [8pts] Make a simulation of the a bouncing ball with the following characterstics.**
\begin{align}
m &=1kg, p_0=(0, 1.5), v_0=(0.3, -0.1), \gamma=0.8,
\end{align}
where $p_0$ and $v_0$ are the initial position and velocity, respectively. $\gamma$ is the coefficient of resititution of the ball. Use $\Delta t=0.001$ sec and simulate the bouncing ball during 3 sec. (Note: No frictional loss)

In [ ]:
def simulate_ball(mass, coeff_rest, init_pos, init_vel, delta_t, N=3000):
  g = -9.8
  p_hist = np.zeros((N, 2))
  v_hist = np.zeros((N, 2))
  t_hist = np.zeros(N)
  p_hist[0, :] = init_pos
  v_hist[0, :] = init_vel
  for i in range(1, N):
      # Fill your code: Add velocity update with acceleration

      if p_hist[i-1, 1]<=0 and v_hist[i, 1]<0:
         # Fill your code: Sudden velocity change when the ball hits the ground


      # Fill your code: Update ball position


      t_hist[i] = i*delta_t;
  return p_hist, v_hist, t_hist

In [ ]:
# Fill your code: set up proper parameters and initial state to call the 'simulate_ball' function


[p_hist, v_hist, t_hist] = simulate_ball(mass, coeff_rest, init_pos, init_vel, delta_t, N)

In [ ]:
# Bouncing ball visualization: No need to change
def plot_bouncingball(p_history, t_history, num_frames= 100):
  fig= plt.figure(figsize=(10,10))
  ax = plt.subplot(1,1,1)
  wall1, = ax.plot([-2, 2], [0, 0], 'b', lw=1)
  ball, = ax.plot([0], [0], 'ro', markersize=12)

  txt_title = ax.set_title('')

  ax.set_xlim(( -1.5, 1.5))
  ax.set_ylim((-0.5, 2.5))
  txt_title = ax.set_title('')
  interval = len(p_history)//num_frames
  def drawFrame(k):
    k = interval*k
    p0 = p_history[k]

    ball.set_data([p0[0]], [p0[1]])
    txt_title.set_text('t = {:.2f} sec'.format(t_history[k]))
    return ball,
  anim = animation.FuncAnimation(fig, drawFrame, frames=num_frames, interval=interval, blit=True)
  return anim

In [ ]:
# Animation code (No need to change)
anim = plot_bouncingball(p_hist, t_hist, num_frames= 100)
plt.close()
HTML(anim.to_html5_video())

writervideo = animation.FFMpegWriter(fps=60)
anim.save('ballbounding.mp4', writer=writervideo)

In [ ]:
plt.plot(p_hist[:, 0], p_hist[:, 1])
plt.show()

##Q.1(b) [5 pts] **What is the position and velocity of the ball after 2 sec ?**

In [ ]:
# Fill your code: Print the position and velocity at 2 sec


print("pos: {}, vel: {} ".format(p_hist[two_sec_idx], v_hist[two_sec_idx]))

##Q.1(c) [5 pts] **What is the position and velocity of the ball after 2 sec when you use $\Delta_t=0.01$? Explain why the values are different from the result of (a). Put your answers in the following text box.**

In [ ]:
# Fill your code: Simulate the bouncing ball again with new delta t

[p_hist_2, v_hist_2, t_hist_2] = simulate_ball(mass, coeff_rest, init_pos, init_vel, delta_t_2, N = 300)

# Fill your code: Print the position and velocity at 2 sec

print("pos: {}, vel: {} ".format(p_hist_2[two_sec_idx], v_hist_2[two_sec_idx]))

In [ ]:
plt.plot(p_hist_2[:, 0], p_hist_2[:, 1])
plt.plot(p_hist[:, 0], p_hist[:, 1])
plt.show()

## Q.1 (c) Put your explanation here:

# Q.2 [40 pts] **Pinball Simulation**
<img src="https://github.com/DARoSLab/Adv_robot_dyn_ctrl/blob/main/Figures/pinball.png?raw=true" width="500">

In [ ]:
# Wall definition code: No need to change
wall1 = np.array([[0, 0], [0, 2]])
wall1_norm = [1, 0]
wall1_tan = [0, 1]

wall2 = np.array([[0, 2], [1, 2]])
wall2_norm = [0, -1]
wall2_tan = [1, 0]


wall3 = np.array([[1, 2], [1, 1]])
wall3_norm = [-1, 0]
wall3_tan = [0, 1]

wall4 = np.array([[1, 1], [0, 0]])
wall4_norm = [-1/np.sqrt(2), 1/np.sqrt(2)]
wall4_tan = [1/np.sqrt(2), 1/np.sqrt(2)]

walls = [wall1, wall2, wall3, wall4]
wall_norms = [wall1_norm, wall2_norm, wall3_norm, wall4_norm]
wall_tan = [wall1_tan, wall2_tan, wall3_tan, wall4_tan]

## Q.2 (a) [20 pts] Simulate the pin ball motion. Assume the radius of the ball is approximately zero and no frictional loss. Use coefficient of restitution, $\gamma$ = 0.9, initial pos: [0.5, 1.0], and initial velocity: [1.5, 0.5]. Simulate over 5 sec.

In [ ]:
# Collision checking code: No need to change
def point_line_distance(point, line):
  distance = np.linalg.norm(np.cross(line[0]-point, line[1]-point))/np.linalg.norm(line[1]-line[0])
  return distance

def detect_collsion(pos, walls, threshold=0.001):
  dlist = []
  collision_idx = None
  for i, wall in enumerate(walls):
    d = point_line_distance(pos, wall)
    dlist.append(d)
    if d<=threshold:
      collision_idx = i
  return collision_idx

In [ ]:
# Simulate Pinball
def simulate_pinball(coeff_rest, init_pos, init_vel, delta_t, walls, wall_norms, wall_tan, N=3000):
  p_hist = np.zeros((N, 2))
  v_hist = np.zeros((N, 2))
  t_hist = np.zeros(N)

  p_hist[0, :] = init_pos
  v_hist[0, :] = init_vel
  t_hist[0] = 0
  for i in range(1, N):
      v_hist[i, :] = v_hist[i-1, :];

      # Fill your code: Check collision and update velocity
      closest_wall_idx =




      p_hist[i, :] = p_hist[i-1, :] + v_hist[i, :]*delta_t;
      t_hist[i] = i*delta_t
  return p_hist, v_hist, t_hist

In [ ]:
# Fill your code: Simulate pinball
dt_pinball = 0.001


[p_hist_pinball, v_hist_pinball, t_hist_pinball] = simulate_pinball(coeff_rest, init_pos, init_vel, dt_pinball, walls, wall_norms, wall_tan, N_pinball)

In [ ]:
# Visualize Pinball Matplotlib: No need to change
def plot_pinball(q, ts, walls, num_frames= 100):
  fig= plt.figure(figsize=(10,10))
  ax = plt.subplot(1,1,1)
  # m, c, l, g, mu, dt = params
  wall1, = ax.plot([walls[0][0][0], walls[0][1][0]], [walls[0][0][1], walls[0][1][1]], 'b', lw=1)
  wall2, = ax.plot([walls[1][0][0], walls[1][1][0]], [walls[1][0][1], walls[1][1][1]], 'b', lw=1)
  wall3, = ax.plot([walls[2][0][0], walls[2][1][0]], [walls[2][0][1], walls[2][1][1]], 'b', lw=1)
  wall4, = ax.plot([walls[3][0][0], walls[3][1][0]], [walls[3][0][1], walls[3][1][1]], 'b', lw=1)
  ball, = ax.plot([0], [0], 'ro', markersize=12)

  txt_title = ax.set_title('')

  ax.set_xlim(( -1, 2))
  ax.set_ylim((-0.5, 2.5))
  txt_title = ax.set_title('')
  interval = len(q)//num_frames
  def drawFrame(k):
    k = interval*k
    q0 = q[k]
    ball.set_data([q0[0]], [q0[1]])
    txt_title.set_text('t = {:.2f} sec'.format(ts[k]))
    return ball,
  anim = animation.FuncAnimation(fig, drawFrame, frames=num_frames, interval=interval, blit=True)
  return anim

In [ ]:
anim = plot_pinball(p_hist_pinball, t_hist_pinball, walls, num_frames= 100)
plt.close()
HTML(anim.to_html5_video())

# writervideo = animation.FFMpegWriter(fps=60)
# anim.save('pinball.mp4', writer=writervideo)

## Q.2.(b) [10 pts] **What is the position and velocity of the ball after 1, 2, 3 sec ?**


In [ ]:
# Fill your code: Print the position and velocity of the ball @ 1, 2, 3 sec


print("1 sec pos: {}, vel: {} ".format(p_hist_pinball[one_sec_idx], v_hist_pinball[one_sec_idx]))
print("2 sec pos: {}, vel: {} ".format(p_hist_pinball[two_sec_idx], v_hist_pinball[two_sec_idx]))
print("3 sec pos: {}, vel: {} ".format(p_hist_pinball[three_sec_idx], v_hist_pinball[three_sec_idx]))

## Q.2.(c) [10 pts] **Plot the kinetic energy of the ball and explain how the ball losese its energy based on the plot**

In [ ]:
kin_energy = np.zeros(N_pinball)

# Fill your code: Compute kinetic energy of the ball and plot


plt.plot(t_hist_pinball, kin_energy)
plt.show()

## Q.2 (c) Put your explanation here:

# Q.3 [40 pts] Swing Stick
<img src="https://github.com/DARoSLab/Adv_robot_dyn_ctrl/blob/main/Figures/SwingArm.png?raw=true" width="500">

## Q.3.(a) [20 pts] Implement a swing stick simulation.
Use parameters, [$m, I, c, l, \mu, dt$] = [1, 0.05, 0.5, 1.0, 0, 0.001].

Simulate the case when [$\theta_0, \dot{\theta}_0$] = [0, 0] and $F$ is applied at the tip of the arm during 0.05 sec to the vertical direction.

(You do not need to consider gravity in this question.)


In [ ]:
def sim_step(theta, theta_dot, F, tau , params=None):
  m, I, c, l, mu, dt = params
  # Fill your code: implement dynamics to compute the angular acceleration


  # Fill your code: Semi-implciti Euler integration


  return [th, th_dot]

def simulate_stick(theta=0, theta_dot=0, F=50, tau=0, F_duration=0.05, F_start_time=0.0, T=1.0, params=None):
  dt = params[-1]
  ts = np.linspace(0, T, int(T/dt))
  theta_hist = [theta]
  theta_dot_hist = [theta_dot]

  # Fill your code: Implement the case that the external force pushes the tip of the arm
  for i, t in enumerate(ts):




  ts = ts.tolist()
  ts.append(T)
  return [theta_hist, theta_dot_hist, ts]

In [ ]:
# Visualization code: No need to change
def plot_stick(q, ts, params, num_frames= 100):
  fig= plt.figure(figsize=(10,10))
  ax = plt.subplot(1,1,1)
  m, I, c, l, mu, dt = params
  link1, = ax.plot([], [], 'b', lw=10)     # ax.plot returns a list of 2D line objects
  txt_title = ax.set_title('')

  ax.set_xlim(( -2.5, 2.5))    # Canvas size
  ax.set_ylim((-2.5, 2.5))
  txt_title = ax.set_title('')
  interval = len(q)//num_frames

  def drawFrame(k):
    k = interval*k
    q0 = q[k]
    rA = [l*np.cos(q0), l*np.sin(q0)]
    x1 = 0
    x2 = rA[0]
    y1 = 0
    y2 = rA[1]
    link1.set_data([x1, x2], [y1, y2])
    txt_title.set_text('t = {:.2f} sec'.format(ts[k]))
    return link1,

  anim = animation.FuncAnimation(fig, drawFrame, frames=num_frames, interval=interval, blit=True)
  return anim

In [ ]:
# Let's use param: [mass, Inertia, CoM, length, friction coefficient, delta_t]
# Fill your code: Run simulation


[theta_hist, theta_dot_hist, ts] = simulate_stick(theta=0, theta_dot=0, F=50, tau=0, F_duration=0.05, F_start_time=0.0, T=2.0, params=params)

In [ ]:
# Joint position and velocity plots (No need to change)
plt.plot(ts, theta_hist)
plt.plot(ts, theta_dot_hist)
plt.show()

In [ ]:
anim = plot_stick(theta_hist, ts, params)
plt.close()
HTML(anim.to_html5_video())

## Q.3.(b) [5 pts] What is the angular position and velocity after 0.5, 1.0, 1.5 sec?

In [ ]:
# Fill your code: Print the angular position and velocity of the arm @ 1, 2, 3 sec
dt_arm = params[-1]


print("1 sec pos: {}, vel: {} ".format(theta_hist[one_sec_idx], theta_dot_hist[one_sec_idx]))
print("2 sec pos: {}, vel: {} ".format(theta_hist[two_sec_idx], theta_dot_hist[two_sec_idx]))
print("3 sec pos: {}, vel: {} ".format(theta_hist[three_sec_idx], theta_dot_hist[three_sec_idx]))

## Q.3. (c) [15 pts] Compute the torque to hold the position of the stick when $F = 50$N at the three following position ($\dot{\theta} = 0$)
### (i) $\theta=0$, $\theta=\frac{\pi}{4}$, and $\theta=\frac{\pi}{6}$
### (ii) Simulate the robot at the configuration in (i) and check whether the stick moves or not.

In [ ]:
_, _, _, l, _, _ = params

# Fill your code: Compute the torque to hold the motion of the arm
F =

theta =
tau =
print('Torque: {:.2f} Nm'.format(float(tau)))



print('Torque: {:.2f} Nm'.format(float(tau)))



print('Torque: {:.2f} Nm'.format(float(tau)))

In [ ]:
# Test your torque: The arm should not move
[theta_hist, theta_dot_hist, ts] = simulate_stick(theta=theta, theta_dot=0, F=50, tau=tau, F_duration=1.0, F_start_time=0.0, T=1.0, params=params)
anim = plot_stick(theta_hist, ts, params)
plt.close()
HTML(anim.to_html5_video())